In [111]:
# load the packages for the project. 
# Might need more down the road

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf



In [112]:
# get the dataframes we are going to use 

# this is the temperature data
weather = pd.read_csv('C:\\Users\\fderango\\Downloads\\Milestone 1 Dataset\\hr_temp.csv')
#temp.head()
weather.tail()
#temp.shape # 26280, 5



,STATION,DATE,REPORT_TYPE,SOURCE,HourlyDryBulbTemperature
26275,72520514762,2020-01-31T19:53:00,FM-15,7,34.0
26276,72520514762,2020-01-31T20:53:00,FM-15,7,33.0
26277,72520514762,2020-01-31T21:53:00,FM-15,7,33.0
26278,72520514762,2020-01-31T22:53:00,FM-15,7,33.0
26279,72520514762,2020-01-31T23:53:00,FM-15,7,34.0


In [113]:
# this is the second dataframe to use 
# this is the megawatt data 
energy = pd.read_csv('C:\\Users\\fderango\\Downloads\\Milestone 1 Dataset\\hrl_load.csv')
#load.head()
energy.head()

,datetime_beginning_utc,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified
0,2/1/2017 5:00,2/1/2017 0:00,RFC,WEST,DUQ,DUQ,1419.881,True
1,2/1/2017 6:00,2/1/2017 1:00,RFC,WEST,DUQ,DUQ,1379.505,True
2,2/1/2017 7:00,2/1/2017 2:00,RFC,WEST,DUQ,DUQ,1366.106,True
3,2/1/2017 8:00,2/1/2017 3:00,RFC,WEST,DUQ,DUQ,1364.453,True
4,2/1/2017 9:00,2/1/2017 4:00,RFC,WEST,DUQ,DUQ,1391.265,True


In [32]:
# it was saying to use a left join first before i did the dictionary and then combined them 

#frames = [weather,energy]

#data = pd.concat(frames)
#data.head(5)
#data.shape

(52560, 13)

In [114]:
# make a new dataframe from the two dataframes we have 
# make a dictionary first than make the new dataframe  

# question #1 
df = {'Date': weather['DATE'],'mw': energy['mw'], 'Temp': weather['HourlyDryBulbTemperature']}

#this is what we want from both dataframes 
new_df = pd.DataFrame(data=df)
new_df.head()
#new_df.shape

,Date,mw,Temp
0,2017-02-01T00:53:00,1419.881,37.0
1,2017-02-01T01:53:00,1379.505,37.0
2,2017-02-01T02:53:00,1366.106,36.0
3,2017-02-01T03:53:00,1364.453,36.0
4,2017-02-01T04:53:00,1391.265,36.0


In [115]:
#question #2 check the data type and convert date into datetime64 
#new_df.dtypes 
new_df['Date'] = pd.to_datetime(new_df['Date'])
new_df.dtypes

Date    datetime64[ns]
mw             float64
Temp           float64
dtype: object

In [116]:
# Question 3 need to create multiple variables from date such as hour, month, week, year  
#new_df.head()

new_df['year'] = new_df['Date'].dt.year
new_df['month'] = new_df['Date'].dt.month
new_df['day_of_week'] = new_df['Date'].dt.dayofweek
new_df['hour_of_day'] = new_df['Date'].dt.hour
#new_df['day'] = new_df['Date'].dt.day
new_df.head()

,Date,mw,Temp,year,month,day_of_week,hour_of_day
0,2017-02-01 00:53:00,1419.881,37.0,2017,2,2,0
1,2017-02-01 01:53:00,1379.505,37.0,2017,2,2,1
2,2017-02-01 02:53:00,1366.106,36.0,2017,2,2,2
3,2017-02-01 03:53:00,1364.453,36.0,2017,2,2,3
4,2017-02-01 04:53:00,1391.265,36.0,2017,2,2,4


In [117]:
#question 4 make the date column the index 
new_df = new_df.set_index('Date')
new_df

,mw,Temp,year,month,day_of_week,hour_of_day
Date,,,,,,
2017-02-01 00:53:00,1419.881,37.0,2017,2,2,0
2017-02-01 01:53:00,1379.505,37.0,2017,2,2,1
2017-02-01 02:53:00,1366.106,36.0,2017,2,2,2
2017-02-01 03:53:00,1364.453,36.0,2017,2,2,3
2017-02-01 04:53:00,1391.265,36.0,2017,2,2,4
...,...,...,...,...,...,...
2020-01-31 19:53:00,1618.484,34.0,2020,1,4,19
2020-01-31 20:53:00,1580.925,33.0,2020,1,4,20
2020-01-31 21:53:00,1545.354,33.0,2020,1,4,21


In [118]:
# question 5 us linear interpolation for the temperture column 
#new_df['Temp'].isnull().sum()

new_df['Temp'] = new_df['Temp'].interpolate(method='linear')




In [119]:
# checking for missing data in the columns after I did interpolate for temp 

new_df.isnull().sum()

mw             0
Temp           0
year           0
month          0
day_of_week    0
hour_of_day    0
dtype: int64

In [123]:
# Question #6 make a training and testing sets for the explore and modeling 

# do it by date index 
train = new_df.loc['2017-02-01': '2019-12-31']
test = new_df.loc['2020-01-01': '2020-01-31']
#train
#train.shape
#train
test
#train = new_df[:25536] # training set
#test = new_df[25536:] # testing set  
#test.head()


,mw,Temp,year,month,day_of_week,hour_of_day
Date,,,,,,
2020-01-01 00:53:00,1363.428,31.0,2020,1,2,0
2020-01-01 01:53:00,1335.975,29.0,2020,1,2,1
2020-01-01 02:53:00,1296.817,30.0,2020,1,2,2
2020-01-01 03:53:00,1288.403,30.0,2020,1,2,3
2020-01-01 04:53:00,1292.263,31.0,2020,1,2,4
...,...,...,...,...,...,...
2020-01-31 19:53:00,1618.484,34.0,2020,1,4,19
2020-01-31 20:53:00,1580.925,33.0,2020,1,4,20
2020-01-31 21:53:00,1545.354,33.0,2020,1,4,21
